## Homework 09:  Parallel Programming 02

## Due Date: Friday Apr 23, 2021, 05:00pm

#### Firstname Lastname: WenxinZhang

#### E-mail: wz2164@nyu.edu

#### Enter your solutions and submit this notebook

---

**Problem 1 (40p)**

In this problem the goal is to calculate the mean and standard deviation of a large list of numbers by using Reduction as one of Collective Operations, see Lecture 11. 


Consider $N = 256000$ random variables uniform on $[0, 1]$, call these $x_0, x_1, \dots, x_{N - 1}$.  


Write an MPI program with $N=16$ processes that outputs the average and standard deviation of $x_0, x_1, \dots, x_{N - 1}$.


To simplify the problem, let one process, say `Process 0`, independently draws $N$ samples uniformly on $[0, 1]$.

How do you explain the results?


**Instructions:** 
Your program should use MPI4PY and collective operations. 
Save your program as 2020_spring_sol09_pr01.py and run it from the terminal as:

```
mpirun -n 16 python 2020_spring_sol09_pr01.py
```


In [1]:
%%writefile 2020_spring_sol09_pr01.py
from mpi4py import MPI
import numpy as np

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

N = 256000

### Let Process 0 independently draw N samples uniformly on [0, 1]
if rank == 0:
    data = np.random.uniform(0, 1, N).astype('f')   
    data = data.reshape(size, int(N/size))    
else:
    data = None

### Each Process charge N/size samples 
recvbuf = np.empty(int(N/size), dtype='f')

### Scatter data to all of the Process
comm.Scatter(data, recvbuf, root=0)
local_sum = sum(recvbuf)

### Calculate the global mean
global_sum = comm.allreduce(local_sum, MPI.SUM)
global_mean = global_sum/N

### Calculate the local sum of the squared difference from the mean
local_sq_diff = sum((num-global_mean)**2 for num in recvbuf)

### Reduce the global sum of the squared differences from the mean to the root Process
global_sq_diff = comm.reduce(local_sq_diff, MPI.SUM, 0)

if rank == 0:
    std_dev = np.sqrt(global_sq_diff/N)
    print(f'- Mean: {global_mean};')
    print(f'- Standard Deviation: {std_dev};')
    print(f'- Standard Deviation calculated by numpy: {np.std(data)};')

Overwriting 2020_spring_sol09_pr01.py


In [2]:
# !mpirun -n 16 python 2020_spring_sol09_pr01.py
# the .py file is runned using Command Line and the output is seen as below

- Mean: 0.5000351471266964;
- Standard Deviation: 0.28842484025216725;
- Standard Deviation calculated by numpy: 0.28842484951019287;


---
**Problem 2 (60p)**

In this problem the goal is to demonstrate how one can use a Domain Decomposition and  Collective Operations. 

Consider the exponential distribution $X \sim \textrm{Exp}(1)$ with the unit mean. Find numerical approximations of moments of the exponential random variable. 

That is, for a random variable $X$ with the distribution $f(x) = e^{-x}$ for $x \geq 0$, compute the first $15$ moments, where the $k$-th moment is defined as:
$$I_k = \int_{0}^{\infty} x^k f(x) dx.$$


Your program should use MPI parallel collective instructions, where the integration is performed in parallel over $N=16$ processes, over a finite range $[0, M)$, where $M=1000$, with $N = 16$ partitions and $1000$ increments per partition, see Lecture 10 and 11.

Provide evaluations of $J_1, J_2, \dots, J_{15}$, where $$J_k = \int_{0}^{M} x^k f(x) dx.$$


**Instructions:** 

Save your program as 2020_sol09_pr02.py; and run it from the terminal as:

```
mpirun -n 16 python 2020_spring_sol09_pr02.py
```


**Bonus Question (10 points):** 

What is the value of $I_k$, as a function of $k$? How can it be derived?

In [3]:
%%writefile 2020_spring_sol09_pr02.py
import numpy
import sys
from math import exp
from mpi4py import MPI

power_k_lst = list(range(1, 16))
for power_k in power_k_lst:
    power_k = int(power_k)

    comm = MPI.COMM_WORLD
    rank = comm.Get_rank()
    size = comm.Get_size()

    def calculate_integral(a_r, delta, n, k):
        integral = 0.0
        for j in range(n):
            x = a_r + (j+0.5) * delta
            integral += (x**k) * exp(-x) * delta
        return integral

    global_integral_start = 0.0
    global_integral_end = 1000.0

    local_integral = numpy.zeros(1)
    global_integral = numpy.zeros(1)

    dest = 0

    ### Initialize the value of n only if this is rank 0
    if rank == 0:
        n = numpy.full(1, 1000, dtype=int) 
    else:
        n = numpy.zeros(1, dtype=int)

    ### Broadcast n to all of the Process
    comm.Bcast(n, root=0)

    ### Each partition, we have 1000 increments
    global_delta = (global_integral_end-global_integral_start) / (n*size)  
    local_integral_start = global_integral_start + rank * (n*global_delta)
    local_integral[0] = calculate_integral(local_integral_start, global_delta, n[0], power_k)

    ### Send each partition back to the root Process
    ### Computing the sum across all of the partitions
    # print("Process", rank, "has the partial integral:", local_integral[0])
    comm.Reduce(local_integral, global_integral, MPI.SUM, dest)

    ### Print the result in Process 0
    if rank == 0:
        print('- The Integral Sum when k=', power_k, 'is', global_integral[0])

Overwriting 2020_spring_sol09_pr02.py


In [4]:
# !mpirun -n 16 python 2020_spring_sol09_pr02.py
# the .py file is runned using Command Line and the output is seen as below

- The Integral Sum when k= 1 is 1.0001627047952104
- The Integral Sum when k= 2 is 2.0000001112238226
- The Integral Sum when k= 3 is 5.99999988885252
- The Integral Sum when k= 4 is 23.999999999771028
- The Integral Sum when k= 5 is 120.00000000022848
- The Integral Sum when k= 6 is 719.9999999999999
- The Integral Sum when k= 7 is 5040.000000000001
- The Integral Sum when k= 8 is 40320.00000000004
- The Integral Sum when k= 9 is 362879.9999999999
- The Integral Sum when k= 10 is 3628800.0000000023
- The Integral Sum when k= 11 is 39916799.99999997
- The Integral Sum when k= 12 is 479001600.0000007
- The Integral Sum when k= 13 is 6227020800.000004
- The Integral Sum when k= 14 is 87178291200.0002
- The Integral Sum when k= 15 is 1307674368000.0007

### Bonus

- What is the value of 𝐼𝑘 , as a function of  𝑘 ? How can it be derived?
- Solution: the value of 𝐼𝑘 as a function of k is 0. As we approaches M to infinity, $e^{-x}$ approches 0 accordingly.

In [5]:
%%writefile 2020_spring_sol09_pr02_bonus.py
import numpy
import sys
from math import exp
from mpi4py import MPI

power_k_lst = list(range(1, 16))
for power_k in power_k_lst:
    power_k = int(power_k)

    comm = MPI.COMM_WORLD
    rank = comm.Get_rank()
    size = comm.Get_size()

    def calculate_integral(start, delta, n, k):
        integral = 0.0
        for partition in range(n):
            x = start + (partition+0.5) * delta
            integral += (x**k) * exp(-x) * delta
        return integral

    global_integral_start = 0.0
#     global_integral_end = 1_000_000.0
#     global_integral_end = 5_000_000.0
    global_integral_end = 100_000_000.0


    local_integral = numpy.zeros(1)
    global_integral = numpy.zeros(1)

    dest = 0

    ### Initialize the value of n only if this is rank 0
    if rank == 0:
        n = numpy.full(1, 1000, dtype=int) 
    else:
        n = numpy.zeros(1, dtype=int)

    ### Broadcast n to all of the Process
    comm.Bcast(n, root=0)

    ### Each partition, we have 1000 increments
    global_delta = (global_integral_end-global_integral_start) / (n*size)  
    local_integral_start = global_integral_start + rank * (n*global_delta)
    local_integral[0] = calculate_integral(local_integral_start, global_delta, n[0], power_k)

    ### Send each partition back to the root Process
    ### Computing the sum across all of the partitions
    # print("Process", rank, "has the partial integral:", local_integral[0])
    comm.Reduce(local_integral, global_integral, MPI.SUM, dest)

    ### Print the result in Process 0
    if rank == 0:
        print('- The Integral Sum when k=', power_k, 'is', global_integral[0])

Overwriting 2020_spring_sol09_pr02_bonus.py


- Setting M = 1_000_000.0
- The Integral Sum when k= 1 is 5.236335679261335e-11
- The Integral Sum when k= 2 is 1.6363548997691671e-09
- The Integral Sum when k= 3 is 5.1136090617786475e-08
- The Integral Sum when k= 4 is 1.5980028318058271e-06
- The Integral Sum when k= 5 is 4.9937588493932106e-05
- The Integral Sum when k= 6 is 0.0015605496404353782
- The Integral Sum when k= 7 is 0.04876717626360557
- The Integral Sum when k= 8 is 1.5239742582376739
- The Integral Sum when k= 9 is 47.624195569927316
- The Integral Sum when k= 10 is 1488.2561115602286
- The Integral Sum when k= 11 is 46508.00348625714
- The Integral Sum when k= 12 is 1453375.1089455357
- The Integral Sum when k= 13 is 45417972.15454798
- The Integral Sum when k= 14 is 1419311629.8296247
- The Integral Sum when k= 15 is 44353488432.17577

- Setting M = 5_000_000.0
- The Integral Sum when k= 1 is 6.763278173450203e-64
- The Integral Sum when k= 2 is 1.0567622146015943e-61
- The Integral Sum when k= 3 is 1.6511909603149911e-59
- The Integral Sum when k= 4 is 2.5799858754921736e-57
- The Integral Sum when k= 5 is 4.031227930456521e-55
- The Integral Sum when k= 6 is 6.298793641338314e-53
- The Integral Sum when k= 7 is 9.841865064591116e-51
- The Integral Sum when k= 8 is 1.537791416342362e-48
- The Integral Sum when k= 9 is 2.4027990880349408e-46
- The Integral Sum when k= 10 is 3.7543735750545943e-44
- The Integral Sum when k= 11 is 5.866208711022805e-42
- The Integral Sum when k= 12 is 9.16595111097313e-40
- The Integral Sum when k= 13 is 1.4321798610895516e-37
- The Integral Sum when k= 14 is 2.2377810329524246e-35
- The Integral Sum when k= 15 is 3.496532863988163e-33

- Setting M = 100_000_000.0
- The Integral Sum when k= 1 is 0.0
- The Integral Sum when k= 2 is 0.0
- The Integral Sum when k= 3 is 0.0
- The Integral Sum when k= 4 is 0.0
- The Integral Sum when k= 5 is 0.0
- The Integral Sum when k= 6 is 0.0
- The Integral Sum when k= 7 is 0.0
- The Integral Sum when k= 8 is 0.0
- The Integral Sum when k= 9 is 0.0
- The Integral Sum when k= 10 is 0.0
- The Integral Sum when k= 11 is 0.0
- The Integral Sum when k= 12 is 0.0
- The Integral Sum when k= 13 is 0.0
- The Integral Sum when k= 14 is 0.0
- The Integral Sum when k= 15 is 0.0